In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import requests, datetime

In [14]:
website_url='https://boardgamegeek.com/browse/boardgame/page/'

In [21]:
response = requests.get(website_url)
response.status_code

200

In [20]:
print(type(response))
response.content

<class 'requests.models.Response'>


b'\n<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tvar width = document.documentElement.clientWidth || window.innerWidth;\n\t\t\t\tif (width < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\t// android debugging\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https:

In [5]:
gameRankings = 'https://boardgamegeek.com/browse/boardgame/page/'

In [6]:
rowDomId = 'row_'
rankObjectClass = 'collection_rank'
idObjectClass = 'collection_objectname'
ratingObjectClass = 'collection_bggrating'
price ='collection_shop'

In [7]:
def get_page(pageNo):
    """This is the webscrape input is "Browse" page number, output is a list of lists of items to collect metadata for"""
    records = []
    importDate = datetime.datetime.strftime(datetime.datetime.now().date(), '%Y-%m-%d')
    req = requests.get(gameRankings + str(pageNo))
    # make sure we get back valid 200 responses
    if req.status_code == 200:
        soup = BeautifulSoup(req.text, 'html.parser')
        rows = soup.find_all(id=rowDomId)
        for row in rows:
            listify = []
            # this searches for a TD element with the class == rankObjectClass
            rankTarget = row.find('td', rankObjectClass)
            if rankTarget:
                listify.append(int(rankTarget.text.strip()))
            # this searches for a TD element with the class == idObjectClass it grabs the ID & Name
            idTarget = row.find('td', idObjectClass)
            if idTarget:
                # ID 
                listify.append((idTarget.find('a')['href']).split('/')[2])
                # Name
                listify.append(idTarget.find('a').text)
                # Description - it is the only paragraoph tag in this section - sometimes not populated
                if idTarget.find('p'):
                    listify.append(idTarget.find('p').getText().strip())
                else:
                    listify.append(None)
            # this colects the 3 td's with the bggRating, avgRating, and numVotes they all use a class of collection_bggrating
            rateTarget = row.find_all('td', ratingObjectClass)
            if len(rateTarget) == 3:
                for target in rateTarget:
                    if target.text.strip().replace('.', '',1).isdigit():
                        listify.append(target.text.strip())
                    else:
                        listify.append(None)
            # if some are missing we have no way of knowing which so we must make all null
            else:
                listify = listify + [None, None, None]
            # add the import date
            listify.append(importDate)
            # check to make sure all record elements are present
            if len(listify) == 8:
                records.append(tuple(listify))
            else:
                # check to make sure all records have the proper amount of elements
                raise Exception(f'The following record is too long/short: {listify} does not have match the rankings format: rank, game_id, title, description, bgg_rating, avg_rating, votes, import_date')
        return tuple(records)


In [9]:
if __name__ == "__main__":
    results = get_page(2)
    for i in results:
        print(i)

ValueError: invalid literal for int() with base 10: 'N/A'

In [ ]:
results

In [10]:
get_page(1)

ValueError: invalid literal for int() with base 10: 'N/A'

In [ ]:
ranking_list=[]

for i in range(5):
    i+=1
    get_page(i)
    print(get_page(i))

In [ ]:
 game_ranking=pd.DataFrame(get_page(i),columns=['rank','id','name','description','bggrating','avgrating','numvoter','date'])

In [ ]:
game_ranking.head()